<a href="https://colab.research.google.com/github/aaliGit-1/Data_Science_Colab/blob/main/LoanApprovalPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
dataset_l=pd.read_csv("Training Dataset.csv")
dataset_l.head(10)

->Importing all the necessary libraries such as Numpy and Pandas<br>
->Read the dataset of csv file in order to work on it and save it to the dataframe named "dataset_l"  

In [ ]:
dataset_l.isnull().sum()

This will display how many rows in a column have null values in it<br>
-> isnull(): check each row of a column whether it is null or not<br>
-> sum(): count how many rows in a column is null

In [ ]:
#Handling Missing Values of Numeric Columns(i.e., LoanAmount,Loan_Amount_Term,Credit_History)
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.NaN,strategy="mean")
dataset_l.iloc[:,8:11]=imputer.fit_transform(dataset_l.iloc[:,8:11])
#Handling Missing Values of the object columns(i.e.,Gender,Married,Dependents,Self_Employed)
dataset_l=dataset_l.dropna()
dataset_l

Importing SimpleImputer class from Sci-Kit learn Library<br>
-> SimpleImputer() method: convert the missing values (i.e., NaN) into the defined strategy (i.e., mean) Conversion is fone into numeric columns only<br>
-> fit_transform() method: It is the combination of both fit() method which helps to compute the mean of a column and transform() method is used to apply that mean to its respective column. So, instead to write the two functions seperately it fit_transform() method is being called<br>
-> dropna() method: Delete all the rows which contains NULL values

In [ ]:
#Encode Categorical Variables
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
dataset_l['Gender']=le.fit_transform(dataset_l['Gender'])
dataset_l['Married']=le.fit_transform(dataset_l['Married'])
dataset_l['Education']=le.fit_transform(dataset_l['Education'])
dataset_l['Self_Employed']=le.fit_transform(dataset_l['Self_Employed'])
dataset_l['Loan_Status']=le.fit_transform(dataset_l['Loan_Status'])
dataset_l

Importing LabelEncoder class from Sci-Kit learn library<br>
-> Creating an instance (i.e., object) of LabelEncoder class and named it "le"<br>
-> Here, all the Categorical Values such as "Gender", "Married", "Education", "Self_Employed" and "Loan_Status" are being converted into numeric values (i.e., 0 & 1) because ML model only understand numeric value<br>
-> display the dataset_l

In [ ]:
#Encoding the 'Dependents' Categorical Variables
dataset_l['Dependents']=pd.to_numeric(dataset_l['Dependents'],errors='coerce').fillna(4).astype(int)
dataset_l['Dependents']

-> to_numeric() method: Convert the column into numeric value and errors='coerce' means that all the non-numeric values replce into NaN<br>
-> fillna(4) method: fill the NULL values places to the defined number as passed in the parenthesis (i.e., 4) <br>
-> astype(int) method: change the dataype into that datatype which is passed through the parenthesis (i.e., int)<br>
-> display the 'Dependent' column

In [ ]:
#Encoding the 'Property_Area' Categorical Variables
propertyarea_dummy=pd.get_dummies(dataset_l['Property_Area'],dtype=int)
dataset_l=pd.concat([dataset_l,propertyarea_dummy],axis=1)
dataset_l.drop('Property_Area',axis=1,inplace=True)
dataset_l

-> get_dummies() method: create the different columns for each category value present in a column and return 1 to a certain column and rest 0 for a particular value in a row<br>
-> dtype property is to return int type value<br>
-> concat() method: add those additional columns in the main dataset and axis=1 refer to columnwise concatenation<br>
-> drop() method: delete the column 'Property_Area' as it is divided into different columns so no need of the column<br>
-> display the dataset

In [ ]:
#Handling outliers for 'ApplicantIncome' Column
def handling_outliers(dataset):
  quantile_25=dataset['ApplicantIncome'].astype(float).quantile(0.25)
  quantile_75=dataset['ApplicantIncome'].astype(float).quantile(0.75)
  iqr=quantile_75-quantile_25
  lower_percentile = quantile_25 - 1.5 * iqr
  upper_percentile = quantile_75 + 1.5 * iqr

  dataset = dataset[dataset['ApplicantIncome'].astype(float) >= lower_percentile]
  dataset = dataset[dataset['ApplicantIncome'].astype(float) <= upper_percentile]

  return dataset
dataset_l=handling_outliers(dataset_l)
dataset_l

Outliers are data points that differ significantly from the majority of the other data. It is being handled so that dataset can be inside a certain region in order to analyze the ML model.<br>
Calculates bounds based on the Inter Quantile Range to identify potential outliers, and filters out any values that fall outside the bounds. This process helps clean the dataset by removing unusually high or low 'ApplicantIncome' values.<br>
>-> define a function named 'handling_outliers' where dataset as a parameter is being passed<br>
-> quantile_25: The 25th percentile (or 1st quartile) of the 'ApplicantIncome' column<br>
-> quantile_75: The 75th percentile (or 3rd quartile) of the 'ApplicantIncome' column<br>
->Both values are converted to float type before quantile calculation<br>
-> The 'iqr' is the difference between the 75th and 25th percentiles. It represents the middle 50% of the data and is often used for detecting outliers<br>
-> The lower bound is calculated as 1.5 times the IQR below the 25th percentile.
The upper bound is calculated as 1.5 times the IQR above the 75th percentile.
These boundaries are typical for outlier detection<br>
-> The dataset is filtered to keep only the rows where 'ApplicantIncome' is within the calculated lower and upper bounds, effectively removing the outliers<br>
-> The function returns the dataset after removing the outliers<br>

Now, 'dataset_l' is going to use the function 'handling_outliers'

In [ ]:
#taking the dependent variable out
y=dataset_l.pop('Loan_Status')
y

Taking th edependent (i.e., target) variabel out

In [ ]:
#Transform into same scale
from sklearn.preprocessing import StandardScaler
dataset_l.drop('Loan_ID',axis=1,inplace=True)
sc=StandardScaler()
X=sc.fit_transform(dataset_l)
X

Importing the StandardScaler class from sklearn library<br>
-> deletimg the Loan_ID<br>
-> creating an instance of StandardScaler class named 'sc' which will help to standardizes the columns of a dataset so that they have a mean of 0 and a standard deviation of 1<br>
-> fit_transform() method will help to apply the StandardScaler to all the columns of a dataset<br>
-> display 'X'



In [ ]:
#Train-Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=0)
X_train

-> importing the train_test_split class from Sci-Kit learn library<br>
-> train_test_split() method is used to split the dataset into training dataset and testing dataset. Here, parameters such as

*   'train_size' is used to declare what proportion of dataset is being used as train set (i.e., 80% is used as train set remaing 20% is used for test set)
*   'random_state' is used to ensures that the same split occurs every time the code is run

-> display the X_train set i.e., train set of independent variable

In [ ]:
#display the X_test set i.e., test set of independent variable
X_test

In [ ]:
#display the y_train set i.e., train set of dependent variable
y_train

In [ ]:
#display the y_test set i.e., test set of dependent variable
y_test

In [ ]:
#Creating Model
from sklearn.linear_model import LogisticRegression
#Train the Logistic Regression model
model=LogisticRegression()
model.fit(X_train,y_train)
#Make predictions
y_pred=model.predict(X_test)
y_pred

-> Import the LogisticRegression class from Sci-Kit learn library of linear_model module<br>
-> Logistic Regression is being used to predicts the probability of a binary outcome (0 or 1) based on input features and instance of LogisticRegression is being created named 'model'<br>
-> fit() method: train the logistic regression model on the training data<br>
-> predict() method: generate predictions based on the test data and save it to 'y_pred'<br>
-> display 'y_pred'

In [ ]:
#Evaluate The Model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy=accuracy_score(y_test,y_pred)
conf_matrix=confusion_matrix(y_test,y_pred)
class_report=classification_report(y_test,y_pred)
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')

-> Import the required class for evaluating the model<br>
-> accuracy_score() method is used to check how accurate is the prediction of the ML model <br>
-> confusion_matrix() method: evaluate the performance of the model by comapring the predicted class labels with the actual class labels<br>
-> classification_report() method: creates a classification summary by displaying the 'Precision', 'Recall', 'F1_score' and 'Support' and provides where areas of improvement need to look.

In [ ]:
#Set display options to show max rows
pd.set_option('display.max_rows',None)
#Create the DataFrame with true and predicted labels
result_df=pd.DataFrame({'True Label':y_test,'Predicted Label':y_pred})
result_df.head(10)

-> Here, I have seperated the DataFrame into 'True Labels' and 'Predicted Labels' and named it as 'result_df' and display it